In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd drive/MyDrive/project-m3-notanagi/notebooks/sft_training

/content/drive/MyDrive/project-m3-notanagi/notebooks/sft_training


In [3]:
!pip install sentencepiece
!pip install -U bitsandbytes
!pip install -U git+https://github.com/huggingface/transformers.git 
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 9.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-sa7pvrvw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-sa7pvrvw
  Resolved https://github.com/huggingface/transformers.git to commit 8f093fb799246f7dd9104ff44728da0c53a9f67a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 10.6 MB/s eta 0:00:0

In [2]:
import sys
MODULE_PATHS = ["../../modules"]
for module_path in MODULE_PATHS:
    if module_path not in sys.path:
        sys.path.append(module_path)

import json


import torch
from fine_tuning import build_finetuning_hf_dataset

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import BitsAndBytesConfig


%load_ext autoreload
%autoreload 2


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

# Loading model for training using LoRa

In [ ]:
model_id = "google/flan-t5-large"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto")

In [4]:
from peft import TaskType

#model.gradient_checkpointing_enable()
#model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, config)

# print number of trainable parameters
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 787,868,672 || trainable%: 0.5989059049678777


In [5]:
# Verifying the datatypes.
dtypes = {}
for _, p in model.named_parameters():
    dtype = p.dtype
    if dtype not in dtypes:
        dtypes[dtype] = 0
    dtypes[dtype] += p.numel()
total = 0
for k, v in dtypes.items():
    total += v
for k, v in dtypes.items():
    print(k, v, v / total)

torch.float32 787868672 1.0


In [6]:
max_input_length = 512
# change max input length of tokenizer to max_input_length
tokenizer.model_max_length = max_input_length
with open("../../submission_items/gen_dataset_NotAnAGI.json", "rb") as reader:
    dataset = build_finetuning_hf_dataset(json.load(reader), tokenizer=tokenizer, max_input_length=max_input_length)

Filter:   0%|          | 0/10647 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (737 > 512). Running this sequence through the model will result in indexing errors


dropped examples:  1721


Map:   0%|          | 0/8926 [00:00<?, ? examples/s]

Map:   0%|          | 0/8926 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8479
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 447
    })
})

In [8]:
dataset["train"][1]["labels"]

tensor([  101,    33,   787,    24,     8,  1612, 10356,   358,  2284,  5354,
         5096,     7,    13,  2850,     6,    84,   598,    34,  1416,    44,
          334,   386, 12096,  2850,    16,     8,   787,  1448,     5,    86,
           48,   495,     6,    62,    33,   787,     8,  1448,    96,  3007,
         7310,  1280,   264,     6,    62,   174,    12,  1099,     8,   826,
         5354,  5096,     7,    10,     3,    18,  8524,   115,     3,    18,
            3, 21207,     3,    18,     3,   115,   115,    40,     3,    18,
            3,  2296,   242,   284,    13,   175,  5354,  5096,     7,     6,
           62,   174,    12, 29216,     8, 15834,    13,    34, 12770,    12,
         1612,   301,     3,   390,    30,     8,   787,  9551,  2197,     5,
          101,    54,   169,     8,  3741,  3356,    13, 15834,    12,   103,
           48,    10,  1514,  3229,   276,   599,     2,  6327,     2,   519,
           18,  5096,  9175,   434,     2,    61,  3274,   276, 

In [9]:
len(dataset["train"][1]["labels"])

512

# Before training

In [12]:
def generate_example(gen_model, tokenizer, dataset, num_examples=5):
    for i in range(num_examples):
        example_input = dataset["test"][i]["input_ids"].to("cuda")
        example_label = dataset["test"][i]["labels"].to("cuda")

        output = gen_model.generate(
            input_ids=example_input.unsqueeze(0),
            max_new_tokens=1000,
            temperature=0.9,
            top_p=0.9,
            num_return_sequences=1,
            repetition_penalty=100.0

        )

        print("RESULT: ", i)
        print(tokenizer.decode(output[0], skip_special_tokens=True))
        # decode input and label
        print("INPUT: ", i)
        print(tokenizer.decode(example_input, skip_special_tokens=True))
        print("LABEL: ", i)
        print(tokenizer.decode(example_label, skip_special_tokens=True))
        print("--------------------------------------------------------")

generate_example(model, tokenizer, dataset, num_examples=5)

RESULT:  0
The redshift of galaxies with a very high redshift is about 5. So the galaxy has to be at least 5 times brighter than the Earth. If the R filter is used, then there are no signs of stellar formation. Thus, it can be concluded that (B).
INPUT:  0
Question: We look for signs of the stellar formation in very old galaxies with the redshift of about 5. So one wants to observe the H alpha emission line in the gas clouds made of ionised hydrogen. The R filter (covering the wavelength range from 6000 to 7000 Angström) seems to be the right filter to use. Unfortunately, there is nothing in the images. Why? (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha line in an ultraviolet filter. (B) Because of the shift towards longer wavelengths at high redshift, one has to observe the H alpha line in an infrared filter. (C) Probably the R filter is dirty, one has to clean it. (D) The galaxies at very high redshift are not brigh

In [10]:
from transformers import DataCollatorForSeq2Seq
training_args = TrainingArguments(
    #auto_find_batch_size=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-3,
    #learning_rate=5e-5,
    fp16=False,
    save_total_limit=4,
    logging_steps=5,
    eval_steps=100,
    output_dir="./outputs",
    save_strategy='epoch',
    #optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.1,
)
class ConversationTrainer(Trainer):    
    def compute_loss(self, model, inputs, return_outputs=False):
        assert return_outputs == False, "we did not implement return_outputs=True in RewardTrainer.compute_loss"
        
        #print("inputs: ", inputs)
        input_ids = inputs["input_ids"]
        labels = inputs["labels"]
        #print("labels: ", labels)
        print("len inputs: ", input_ids.shape)
        print("len labels: ", labels.shape)
        loss, outputs = Trainer.compute_loss(self, model, inputs, return_outputs=True)
        print("loss: ", loss)
        print("outputs: ", outputs)
        wefwefw
        return loss



        


# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding="max_length",
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


trainer = Trainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_args,
    data_collator=data_collator
)



In [11]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
5,1.981500
10,1.981500
15,2.104500
20,2.087000
25,1.935300
30,1.745200
35,2.106100
40,1.778500
45,1.805100
50,1.945200


TrainOutput(global_step=2119, training_loss=1.6154496970183447, metrics={'train_runtime': 4971.0668, 'train_samples_per_second': 1.706, 'train_steps_per_second': 0.426, 'total_flos': 1.965807713334067e+16, 'train_loss': 1.6154496970183447, 'epoch': 1.0})

# After training

In [13]:
def generate_example(gen_model, tokenizer, dataset, num_examples=5):
    for i in range(num_examples):
        example_input = dataset["test"][i]["input_ids"].to("cuda")
        example_label = dataset["test"][i]["labels"].to("cuda")

        output = gen_model.generate(
            input_ids=example_input.unsqueeze(0),
            max_new_tokens=1000,
            temperature=0.9,
            top_p=0.9,
            num_return_sequences=1,
            #repetition_penalty=100.0

        )

        print("RESULT: ", i)
        print(tokenizer.decode(output[0], skip_special_tokens=True))
        # decode input and label
        print("INPUT: ", i)
        print(tokenizer.decode(example_input, skip_special_tokens=True))
        print("LABEL: ", i)
        print(tokenizer.decode(example_label, skip_special_tokens=True))
        print("--------------------------------------------------------")

generate_example(model, tokenizer, dataset, num_examples=5)

RESULT:  0
The correct answer is: All of the above.
INPUT:  0
Question: What standard minimal information about a dataset is needed in order to understand, interpret and use it? • Subject, methods and brain location • Type and format of data • Contributors, license and access to the data • All of the above Answer: 
LABEL:  0
The correct answer is: All of the above. Subject, methods, and brain location provide context and understanding of the data. The type and format of data determine what analyses can be performed on the data. Contributors, license, and access to the data determine if the data can be legally and ethically used. Therefore, all of the above are needed to understand, interpret, and use a dataset.
--------------------------------------------------------
RESULT:  1
The correct answer is A. when a set of variables in the feature space are not linearly independent. This is because feature degeneracy is a problem in the field of feature analysis. The correct answer is A. when

# Checkpoint 1

In [ ]:
from peft import PeftModel, PeftConfig
peft_model_id = "../../../outputs/checkpoint-632"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
model1 = PeftModel.from_pretrained(model, peft_model_id)
model1.eval()

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=768, out_features=768, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B):

In [ ]:
def generate_example(gen_model, tokenizer, dataset, num_examples=5):
    for i in range(num_examples):
        example_input = dataset["test"][i]["input_ids"].to("cuda")
        example_label = dataset["test"][i]["labels"].to("cuda")

        output = gen_model.generate(
            input_ids=example_input.unsqueeze(0),
            max_new_tokens=1000,
            temperature=0.9,
            top_p=0.9,
            num_return_sequences=1,
            repetition_penalty=100.0

        )

        print("RESULT: ", i)
        print(tokenizer.decode(output[0], skip_special_tokens=True))
        # decode input and label
        print("INPUT: ", i)
        print(tokenizer.decode(example_input, skip_special_tokens=True))
        print("LABEL: ", i)
        print(tokenizer.decode(example_label, skip_special_tokens=True))
        print("--------------------------------------------------------")

generate_example(model1, tokenizer, dataset, num_examples=5)

RESULT:  0
The correct answer is (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha emission line in an ultraviolet filter. This means that there are no visible signs of stellar formation in very old galaxie with the redshift of about 5. Therefore, we need to use the R filter covering the wavelength range from 6000–7000 Angström.
INPUT:  0
Question: We look for signs of the stellar formation in very old galaxies with the redshift of about 5. So one wants to observe the H alpha emission line in the gas clouds made of ionised hydrogen. The R filter (covering the wavelength range from 6000 to 7000 Angström) seems to be the right filter to use. Unfortunately, there is nothing in the images. Why? (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha line in an ultraviolet filter. (B) Because of the shift towards longer wavelengths at high redshift, one has to observe 

# checkpoint 4

In [ ]:
from peft import PeftModel, PeftConfig
peft_model_id = "../../trained_peft"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
model4 = PeftModel.from_pretrained(model, peft_model_id)
model4 = model4.eval()

In [ ]:
generate_example(model4, tokenizer, dataset, num_examples=5)

RESULT:  0
The correct answer is (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha emission line in an ultraviolet filter. This means that there are no visible signs of stellar formation in very old galaxie with the redshift of about 5. Therefore, we need to use the R filter covering the wavelength range from 6000–7000 Angström.
INPUT:  0
Question: We look for signs of the stellar formation in very old galaxies with the redshift of about 5. So one wants to observe the H alpha emission line in the gas clouds made of ionised hydrogen. The R filter (covering the wavelength range from 6000 to 7000 Angström) seems to be the right filter to use. Unfortunately, there is nothing in the images. Why? (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha line in an ultraviolet filter. (B) Because of the shift towards longer wavelengths at high redshift, one has to observe 

# After PPO training

In [ ]:
from peft import PeftModel, PeftConfig

model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)


peft_model_id = "../../models/assistant-ppo"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
model_ppo = PeftModel.from_pretrained(model, peft_model_id)
model_ppo = model_ppo.eval()

In [ ]:
generate_example(model_ppo, tokenizer, dataset, num_examples=5)

RESULT:  0
(A) The cosmological shift towards smaller wavelengths at high redshift is a cosmological shift.
INPUT:  0
Question: We look for signs of the stellar formation in very old galaxies with the redshift of about 5. So one wants to observe the H alpha emission line in the gas clouds made of ionised hydrogen. The R filter (covering the wavelength range from 6000 to 7000 Angström) seems to be the right filter to use. Unfortunately, there is nothing in the images. Why? (A) Because of the cosmological shift towards smaller wavelengths at high redshift, one has to observe the H alpha line in an ultraviolet filter. (B) Because of the shift towards longer wavelengths at high redshift, one has to observe the H alpha line in an infrared filter. (C) Probably the R filter is dirty, one has to clean it. (D) The galaxies at very high redshift are not bright enough to be observed. Answer: Let’s think step by step: Answer: 
LABEL:  0
At high redshift, there is a cosmological shift towards longe